In [1]:
from math import sqrt
from scipy import stats
from pydataset import data
from datetime import datetime


from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import sklearn.metrics

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from env import host, user, password, sql_connect
from wrangle import wrangle_telco, wrangle_zillow

## Exercise II 

#### Question 1: Acquire the telco_churn database into a pandas dataframe selecting the customer_id, monthly_charges, tenure, and total_charges columns. While filtering for customers that are 2 year contract customers.

In [2]:
##built sql query to select the appropriate columns and filter for 2 year customers below

sql_query = '''
Select customer_id, monthly_charges, tenure, total_charges
from customers
where contract_type_id = 3;
'''
## use sql_connect function from env to connect to telco_churn and pull the table made from
## the query into a pandas dataframe

df = pd.read_sql(sql_query, sql_connect('telco_churn'))

In [3]:
df.head() ##displaying dataframe

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.8
2,0016-QLJIS,90.45,65,5957.9
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75


In [4]:
df.shape  ##displaying dataframe shape (rows, columns)

(1695, 4)

#### Question 2: Using your acquired Telco data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

In [5]:
df.describe().T  ##looking at numerical statistics seems some columns are missing lets check
## it out

,count,mean,std,min,25%,50%,75%,max
monthly_charges,1695.0,60.770413,34.678865,18.4,24.025,64.35,90.45,118.75
tenure,1695.0,56.735103,18.209363,0.0,48.000,64.00,71.00,72.00


In [6]:
df.info() ## the total_charges column is an object we should look into changing this so 
## it matches the other charges column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


In [7]:
df.sort_values(by = 'total_charges', ascending = True) 
##seeing we have blank values in total charges

,customer_id,monthly_charges,tenure,total_charges
524,3213-VVOLG,25.35,0,
416,2520-SGTTA,20.00,0,
678,4075-WKNIU,73.35,0,
234,1371-DWPAZ,56.05,0,
941,5709-LVOEQ,80.85,0,
...,...,...,...,...
731,4526-EXKKN,24.60,40,973.95
442,2675-IJRGJ,19.65,53,978
582,3521-HTQTV,26.10,34,980.35
714,4361-FEBGN,20.15,48,982.95


In [8]:
df.total_charges = df.total_charges.str.replace(' ', '0').astype(float) 

## deal with the blank values by replacing them with 0's because these
## customers haven't had a full month of tenure yet so their technical
## total_charges is 0 and also turn type to float

In [9]:
df.sort_values(by = 'total_charges', ascending = True)

## making sure they were in fact replaced with 0's

,customer_id,monthly_charges,tenure,total_charges
678,4075-WKNIU,73.35,0,0.00
1293,7644-OMVMY,19.85,0,0.00
716,4367-NUYAO,25.75,0,0.00
234,1371-DWPAZ,56.05,0,0.00
726,4472-LVYGI,52.55,0,0.00
...,...,...,...,...
1679,9924-JPRMC,118.20,72,8547.15
1504,8879-XUAHX,116.25,71,8564.75
1657,9788-HNGUT,116.95,72,8594.40
1649,9739-JLPQJ,117.50,72,8670.10


In [10]:
df.info() 

## checking datatypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.1+ KB


In [11]:
df = df.reset_index()
df.head()

## reseting the index since we pulled a sample of the whole dataframe
## aka we filtered for 2-year customers

,index,customer_id,monthly_charges,tenure,total_charges
0,0,0013-SMEOE,109.70,71,7904.25
1,1,0014-BMAQU,84.65,63,5377.80
2,2,0016-QLJIS,90.45,65,5957.90
3,3,0017-DINOC,45.20,54,2460.55
4,4,0017-IUDMW,116.80,72,8456.75


#### Question 3: End with a wrangle.py file that contains the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values. Name your final function wrangle_telco.


In [12]:
telco_df = wrangle_telco()  ##testing function
telco_df.head()

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.80
2,0016-QLJIS,90.45,65,5957.90
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75


In [13]:
telco_df.info() ##checking datatypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.1+ KB


## Exercise III - Zillow Database

In [14]:
sql_query = '''
select bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
from properties_2017
where propertylandusetypeid = 261;
'''

zillow_df = pd.read_sql(sql_query, sql_connect('zillow'))
zillow_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [15]:
zillow_df.shape

(2152863, 7)

In [16]:
zillow_df.info()   ## looking at data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 115.0 MB


In [17]:
zillow_df.describe().T   ##looking at numerical variable statistics

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2152852.0,3.287196,0.954754,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2152852.0,2.230688,0.999280,0.00,2.00,2.00,3.00,32.00
calculatedfinishedsquarefeet,2144379.0,1862.855178,1222.125124,1.00,1257.00,1623.00,2208.00,952576.00
taxvaluedollarcnt,2152370.0,461896.237963,699676.049600,1.00,188170.25,327671.00,534527.00,98428909.00
yearbuilt,2143526.0,1960.949681,22.162196,1801.00,1949.00,1958.00,1976.00,2016.00
taxamount,2148421.0,5634.865978,8178.910249,1.85,2534.98,4108.95,6414.32,1337755.86
fips,2152863.0,6048.377335,20.433292,6037.00,6037.00,6037.00,6059.00,6111.00


In [18]:
zillow_df.isnull().sum()  #checking for null values

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

In [19]:
zillow_df = zillow_df.dropna()  ### given the dataframe we pulled has over 2 million rows

## we are going to drop the null values because it is a small percentage of the entire dataframe

In [20]:
zillow_df.isnull().sum()  ##checking to see if null rows were removed

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
dtype: int64

In [21]:
zillow_df.shape  ##displaying shape of dataframe (rows, columns)

(2140235, 7)

In [22]:
zillow_df.head()   ##displaying dataframe

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0


In [23]:
zillow_df = zillow_df.reset_index()

In [24]:
zillow_df = zillow_df.drop(columns = 'index')  ##dropping old index column
zillow_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
1,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
2,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
3,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
4,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0


In [25]:
zillow_test = wrangle_zillow()  ##testing function
zillow_test.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
1,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
2,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
3,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
4,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0
